In [ ]:
# set up imports
import os
import sys
__file__ = os.getcwd()
proj_dir =  os.path.dirname(os.path.realpath(__file__))
sys.path.append(proj_dir)
utils_dir = os.path.join(proj_dir,'utils')
sys.path.append(utils_dir)
analysis_dir = os.path.join(proj_dir,'analysis')
analysis_utils_dir = os.path.join(analysis_dir,'utils')
sys.path.append(analysis_utils_dir)
agent_dir = os.path.join(proj_dir,'model')
sys.path.append(agent_dir)
agent_util_dir = os.path.join(agent_dir,'utils')
sys.path.append(agent_util_dir)
experiments_dir = os.path.join(proj_dir,'experiments')
sys.path.append(experiments_dir)
df_dir = os.path.join(proj_dir,'results/dataframes')

In [ ]:
from model.Subgoal_Planning_Agent import *
import utils.blockworld as bw
import utils.blockworld_library as bl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import model.utils.decomposition_functions
from analysis.sequence_selection import *

In [ ]:
#inline plots
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
plt.rcParams["figure.figsize"] = (20,7)
plt.rcParams.update({'font.size': 22})

In [ ]:
#display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 20)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.min_rows', 6)

Let's load the results of the experiment

In [ ]:
df_paths = ['for human experiments/Astar complete sequences 5e for best worst lookahead.pkl']

In [ ]:
#load all experiments as one dataframe
df = pd.concat([pd.read_pickle(os.path.join(df_dir,l)) for l in df_paths])
print("Loaded dataframe")

In [ ]:
df

In [ ]:
# compute sequences with best overall sequence
sequences_df = pd.DataFrame()

for agent in df['lower level: agent_type'].unique():
    for world in df['world'].unique():
        list_of_list_of_sequences = df[(df['world'] == world) & (df['lower level: agent_type'] == agent)]['_all_sequences'].to_list()
        try:
            best_name, worst_name, best_cost, worst_cost = get_best_and_worst_sequence(list_of_list_of_sequences)
        except TypeError:
            best_name = worst_name = _best_cost = worst_cost = None
        sequences_df = sequences_df.append({'agent': agent, 'world': world, 'best_name': best_name, 
                             'worst_name': worst_name, 'best_cost': best_cost, 
                             'worst_cost':worst_cost},
                           ignore_index = True)


In [ ]:
sequences_df

In [ ]:
# compute sequences with most divergent pair
sequences_df = pd.DataFrame()

for agent in df['lower level: agent_type'].unique():
    for world in df['world'].unique():
        list_of_list_of_sequences = df[(df['world'] == world) & (df['lower level: agent_type'] == agent)]['_all_sequences'].to_list()
        try:
            best_name, worst_name, delta = get_most_divergent_pair_sequence(list_of_list_of_sequences)
        except TypeError:
            best_name = worst_name = delta = None
        sequences_df = sequences_df.append({'agent': agent, 'world': world, 'best_name': best_name, 
                             'worst_name': worst_name, 'delta': delta},
                           ignore_index = True)


In [ ]:
sequences_df

Save to file

In [ ]:
OUT_DIR = "/Users/felixbinder/Desktop"

In [ ]:
sequences_df.to_json(OUT_DIR+"/best_worst_subgoal_seqs.json")